<!-- Container Image -->

image: node03.st:5000/pyspark-hdfs-jupyter:eabraham-373705

## Task 1 (1 point) Create Jupyter’s Deployment and Service.

Requirements:
а. Guaranteed QoS. Resources amount - 1 CPU, 2 Gb RAM
b. Replicas number - 1.
c. Pod has to contain the following label - jupyter: lab0
d. Deployment name: lab0-jupyter
e. Service name: lab0-jupyter-service
f. Service has to forward requests only to this deployment’s pod. Ports number for traffic forwarding - 1.
g. Service type - NodePort



Based on the above requirements, here's the Kubernetes deployment and service manifest for Jupyter:

Also,

Guaranteed Quality of Service (QoS) is a concept in Kubernetes that ensures a certain level of resources for Pods. Kubernetes has three QoS classes: Guaranteed, Burstable, and BestEffort.

In the context of your manifest, Guaranteed QoS is achieved by setting the same values for both resource requests and resource limits in the Pod spec.

In [ ]:
# apiVersion: apps/v1
# kind: Deployment
# metadata:
#   namespace: <your-namespace>
#   name: lab0-jupyter
#   labels:
#     jupyter: lab0
# spec:
#   replicas: 1
#   selector:
#     matchLabels:
#       jupyter: lab0
#   template:
#     metadata:
#       labels:
#         jupyter: lab0
#     spec:
#       containers:
#       - name: jupyter-container
#         image: node03.st:5000/pyspark-hdfs-jupyter:<your-namespace>
#         ports:
#         - containerPort: 8080
#         resources:
#           limits:
#             cpu: "1"
#             memory: "2Gi"
#           requests:
#             cpu: "1"
#             memory: "2Gi"
# ---
# apiVersion: v1
# kind: Service
# metadata:
#   name: lab0-jupyter-service
#   namespace: <your-namespace>
# spec:
#   type: NodePort
#   selector:
#     jupyter: lab0
#   ports:
#   - protocol: TCP
#     port: 80
#     targetPort: 8080

You can create this file with your favorite editor, in your preferred directory, and copy it to your cluster:

scp lab0-jupyter.yaml <your-namespace>@gateway.st:~/lab0-jupyter.yaml

And then apply the updated manifest

kubectl apply -f lab0-jupyter.yaml


## 2. (2 points) Modify previous Deployment and Service to face following conditions:

а. Jupyter has to listen on 8282 port (you can set it up using args).
b. Jupyter’s token has to match your gateway.st password.
c. Service has to receive requests on port 80 and forward them on 8282 port .
d. Deployment and Service names have to stay the same!

To modify the Deployment and Service to meet the new requirements, you need to update your lab0-jupyter.yaml

In [ ]:
# apiVersion: apps/v1
# kind: Deployment
# metadata:
#   namespace: <your-namespace>
#   name: lab0-jupyter
#   labels:
#     jupyter: lab0
# spec:
#   replicas: 1
#   selector:
#     matchLabels:
#       jupyter: lab0
#   template:
#     metadata:
#       labels:
#         jupyter: lab0
#     spec:
#       containers:
#       - name: jupyter-container
#         image: node03.st:5000/pyspark-hdfs-jupyter:<your-namespace>
#         ports:
#         - containerPort: 8282
#         args: ["--port", "8282", "--NotebookApp.token=<your-own-password>"]
#         args: ["jupyter", "notebook", "--port", "8282", "--NotebookApp.token='pass1word'"]

#         resources:
#           limits:
#             cpu: "1"
#             memory: "2Gi"
#           requests:
#             cpu: "1"
#             memory: "2Gi"
# ---
# apiVersion: v1
# kind: Service
# metadata:
#   name: lab0-jupyter-service
#   namespace: <your-namespace>
# spec:
#   type: NodePort
#   selector:
#     jupyter: lab0
#   ports:
#   - protocol: TCP
#     port: 80
#     targetPort: 8282

Here's what changed:

Updated the containerPort to 8282.
Added args to the container spec to set up Jupyter to listen on port 8282 and use your gateway.st password as the token. Replace <your_gateway_password> with your actual password.
Updated the targetPort of the Service to 8282.
With these changes, your Deployment and Service should now meet the new requirements. 

You can edit with favorite editor, in your preferred directory, and copy updated file it to your cluster:

scp lab0-jupyter.yaml <your-namespace>@gateway.st:~/lab0-jupyter.yaml

And then apply the updated manifest

kubectl apply -f lab0-jupyter.yaml


## 3. (2 points) Create the following ConfigMap (CM):

a. CM name: lab0-jupyter-cm
b. CM has to contain a  file-like key. Key - jupyter_notebook_config.py
c. Content for the jupyter_notebook_config.py key (2 rows): 
c.NotebookApp.trust_xheaders = True
c.NotebookApp.quit_button = False


You can create the ConfigMap using a YAML manifest file

In [ ]:
# apiVersion: v1
# kind: ConfigMap
# metadata:
#   name: lab0-jupyter-cm
#   namespace: <your-namespace>
# data:
#   jupyter_notebook_config.py: |
#     c.NotebookApp.trust_xheaders = True
#     c.NotebookApp.quit_button = False

you can vim or nano lab0-jupyter-cm.yaml to create this file in your cluster

Then apply the manifest

kubectl apply -f lab0-jupyter-cm.yaml

## 4. (3 points) Mount jupyter_notebook_config.py (only this file) from your CM into the Jupyter pod:

а. Mount has to be completed from the CM.
b. Mount path - /home/jovyan/.jupyter/jupyter_notebook_config.py
c. Mount has to be read only.


To mount the jupyter_notebook_config.py from your ConfigMap into the Jupyter pod, you need to update your Deployment manifest to include volumeMounts and volumes sections.

In [ ]:
# apiVersion: apps/v1
# kind: Deployment
# metadata:
#   namespace: <your-namespace>
#   name: lab0-jupyter
#   labels:
#     jupyter: lab0
# spec:
#   replicas: 1
#   selector:
#     matchLabels:
#       jupyter: lab0
#   template:
#     metadata:
#       labels:
#         jupyter: lab0
#     spec:
#       containers:
#       - name: jupyter-container
#         image: node03.st:5000/pyspark-hdfs-jupyter:<your-name-space>
#         ports:
#         - containerPort: 8282
#         args: ["jupyter", "notebook", "--port", "8282", "--NotebookApp.token='pass1word'"]

#         resources:
#           limits:
#             cpu: "1"
#             memory: "2Gi"
#           requests:
#             cpu: "1"
#             memory: "2Gi"
#         volumeMounts:
#         - name: config-volume
#           mountPath: /home/jovyan/.jupyter/jupyter_notebook_config.py
#           subPath: jupyter_notebook_config.py
#           readOnly: true
#       volumes:
#       - name: config-volume
#         configMap:
#           name: lab0-jupyter-cm
# ---
# apiVersion: v1
# kind: Service
# metadata:
#   name: lab0-jupyter-service
#   namespace: <your-name-space>
# spec:
#   type: NodePort
#   selector:
#     jupyter: lab0
#   ports:
#   - protocol: TCP
#     port: 80
#     targetPort: 8282

Apply the updated manifest to your cluster as usual

kubectl apply -f lab0-jupyter-cm.yaml

In the above manifest, the volumeMounts section is added to the container spec, which mounts the ConfigMap's data as a file at the specified mountPath. The subPath field is used to specify the key from the ConfigMap, which in this case is jupyter_notebook_config.py. The readOnly: true option ensures that the mounted file is read-only.

The volumes section is added to the pod spec, which defines a volume named config-volume that references the ConfigMap lab0-jupyter-cm. This volume is then used in the volumeMounts section to mount the data from the ConfigMap.

## 5. (1 point) Save the Jupyter’s pod log with timestamps into the file lab0-jupyter.log


To save the Jupyter's pod logs with timestamps into a file, first, you need to get the name of the Jupyter pod. You can do this by running:

kubectl get pods -l jupyter=lab0

Once you have the pod name, you can save the logs with timestamps by running:

In [ ]:
# kubectl logs pod/(name_of_your_pod) --timestamps > lab0-jupyter.log


## 6. (1 point) Save in file lab0-ls-lah.sh the command to list files in the Jupyter container path /home/jovyan/.jupyter.

You can create the file with nano or vim

nano lab0-ls-lah.sh

To save the command that lists files in the Jupyter container path /home/jovyan/.jupyter in a file called lab0-ls-lah.sh, simply create a file with the following content:


In [ ]:
#!/bin/bash

# kubectl exec -it (name_of_your_pod) -- ls -lah /home/jovyan/.jupyter


Make sure to give the script executable permissions:

chmod +x lab0-ls-lah.sh

Now, you can run the script using:

./lab0-ls-lah.sh


## CLEAN UP REPLICASETS WITH ZERO DESIRED, CURRENT AND READY REPLICSAS

In [ ]:
# kubectl delete replicaset lab0-jupyter-5c55f44675
# kubectl delete replicaset lab0-jupyter-5f48658678
# kubectl delete replicaset lab0-jupyter-65f698fd75